# Knowledge distillation
Suppose that we have a large network (*teacher network*) or an ensemble of networks which has a good accuracy but doesn't fit into memory/runtime requirements. Instead of training a smaller network (*student network*) directly on the original dataset, we can train this network to predict outputs of teacher networks. It turns out that the perfomance could be even better! This approach doesn't help with training speed, but can be quite beneficial when we'd like to reduce the model size for low-memory devices.

* https://www.ttic.edu/dl/dark14.pdf
* [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531)
* https://medium.com/neural-machines/knowledge-distillation-dc241d7c2322

Even the completely different ([article](https://arxiv.org/abs/1711.10433)) architecture can be used in a student model, e.g. you can approximate an autoregressive model (WaveNet) by a non-autoregressive one.

# Task
## 1. Teacher network
Train good enough (teacher) network, achieve >=35% accuracy on validation set of Tiny Imagenet (you can reuse any network from homework part 1 here).

## 2. Student network 
Train small (student) network, achieve 20-25% accuracy, draw a plot "training and testing errors vs train step index"

## 3. Knowledge distillation
![image info](https://miro.medium.com/max/875/1*WxFiH3XDY1-28tbyi4BGDA.png)

At this block you will have to retrain your student network using "knowledge distillation" technique. **Distill teacher network with student network, achieve at least +1% improvement in accuracy over student network accuracy.**

The training procedure is the same as for training the student network from scratch except the loss formulation.

Assume that 
- $z_i$ are logits predicted by the student network at the current step for an input image
- $v_i$ are logits predicted by the (freezed) teacher network
- $y_i$ are one-hot encoded label of the input image
- $p_i = \frac{\exp{z_i}}{\sum_j \exp{z_j}}$ - logits $z_i$ after sofrmax
- $q_i = \frac{\exp{\frac{z_i}{T}}}{\sum_j \exp{\frac{z_j}{T}}}$, where $T$ is softmax temperature
- $r_i = \frac{\exp{\frac{v_i}{T}}}{\sum_j \exp{\frac{v_j}{T}}}$, where $T$ is the same softmax temperature as for $q_i$

The loss for knowledge distillation: $$-\sum_i y_i \log p_i - \alpha \sum_i r_i \log q_i$$

$T$ and $\alpha$ are hyperparameters. 

- There is a good practice of using softmax with high temperature to obtain "soft" distributions, you can start with $T=10$. Check the [post](https://medium.com/mlearning-ai/softmax-temperature-5492e4007f71) with good visualizations on how the temperature affects the softmax output. 
- For $\alpha$ there is the following note in the original [paper](https://arxiv.org/pdf/1503.02531.pdf):

> Since the magnitudes of the gradients produced by the soft targets scale as $1/T^2$ it is important to multiply them by $T^2$ when using both hard and soft targets. This ensures that the relative contributions of the hard and soft targets remain roughly unchanged if the temperature used for distillation is changed while experimenting with meta-parameters.

**Final notes**:
- Please, don't cheat with early-early-early stopping while training of the student network. Make sure, it  converged.
- Logits still carry more information than the probabilities after softmax.
- Don't forget to use your teacher network in 'eval' mode. And don't forget your main objective.

**Future readings**
- ["Born again neural networks"](https://arxiv.org/pdf/1805.04770.pdf) - knowledge distillation may give benefits even when teacher and student networks have the same architecture.
- ["Prune your model before distill it"](https://arxiv.org/pdf/2109.14960.pdf) - pruning of the teacher model before distillation may improve quality of student model